# Microbiome Classification Analysis
After we are done with the pre-processing, we then continue the microbiome classification analysis by using the `Representative Sequences` generated from denoising step. From this analysis, we will get the 'Taxonomic Classification' and 'Phylogenetic Tree'

## Taxonomic Classification
Before we complete our upstream analysis steps, we’ll generate taxonomic annotations for our sequences using the feature-classifier plugin to identify amplicon sequence variants (ASVs) or operational taxonomic units (OTUs). A pre-trained classifier is provided in materials folder under the name `classifier-V34.qza` which was trained on SILVA database using QIIME 2 version 2024.10 specifically. If you use another version of QIIME 2 or want to try another database, you can train a new classifier yourself or find the classifier shared by other users in QIIME 2 forum.

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier ../classifier-V34.qza \
  --i-reads rep-seqs.qza \
  --o-classification silva-taxonomy.qza

We can visualize the taxonomic classification by utilizing `qiime taxa barplot` command. See the interactive barplots in [QIIME 2 View](https://view.qiime2.org).

In [ ]:
qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy silva-taxonomy.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization silva-taxa-bar-plots.qzv

## Building Phylogenetic Tree
QIIME 2 allows you to generate phylogenetic trees de novo with a few different underlying tools. We’ll build a phylogenetic tree from our ASVs using the q2-phylogeny plugin’s align_to_tree_mafft_fasttree action. This action is a pipeline in QIIME 2, which means that it strings together multiple simpler operations that are often performed together to reduce the number of steps that users have to take. Pipelines are used in the same way as other actions.

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

qiime empress community-plot \
  --i-tree rooted-tree.qza \
  --i-feature-table asv-table.qza \
  --m-sample-metadata-file sample-metadata.tsv \
  --m-feature-metadata-file silva-taxonomy.qza \
  --o-visualization tree-visualization.qzv

QIIME Empress helps us explore the hierarchical and evolutionary relationships between features in a dataset. Upload `tree-visualization.qzv` to [QIIME 2 View](https://view.qiime2.org/) to generate a visualization.

## Taxonomy Visualization

We can visualize the abundance taxonomy by using Krona.
You can install the Krona package under QIIME2 environment [q2-krona](https://github.com/kaanb93/q2-krona/) and additional tools to [prepare the data](https://github.com/lokeshbio/Amplicon_course/blob/master/Qiime2_pipeline_melina.md#krona-plots) 

The input data are: 
- Feature-table data in tsv format
- Taxonomy data in tsv format

First, we need to convert the Feature-table and Taxonomy data into tsv format

In [ ]:
qiime tools export --input-path [path]/asv-table.qza --output-path [path]/
biom convert -i [path]/asv-table.biom -o [path]/asv-table.tab --to-tsv

qiime tools export --input-path [path]/silva-taxonomy.qza --output-path [path]/


We can then using [KronaTools](https://github.com/lokeshbio/Amplicon_course/blob/master/KronaTools-2.7.tar) as additional tools to combine the taxonomy and abundance.

Combine the the taxonomic classification of the ASV to their abundance using the 'krona_qiime.py' command from KronaTools. 
This will create text files for each sample in the analysis.                                                                         

In [ ]:
tar xvf KronaTools-2.7.tar
cd KronaTools-2.7
sudo ./install.pl
cd ..

mkdir [path]/txt_files
cd [path]/txt_files
../krona_qiime.py [path]/silva-taxonomy.tsv [path]/asv-table.tab
cd ..

ktImportText [path]/txt_files/* -o [path]/krona.html

Congratulations, you have sucessfully analyze the microbiome abundance. Now continue to the next section of the tutorial: [Statistics Analysis](5_Statistics_Analysis.ipynb),

or you just can back to the main tutorial page [Main Page](1_Metagenomics_Workshop_Module.ipynb)